In [1]:
import pandas as pd
import numpy as np

In [2]:
def carregar_liga(caminho_arquivo):
    try:
        df = pd.read_csv(caminho_arquivo)
        if 'estilo' not in df.columns:
            df['estilo'] = 'equilibrado'
        else:
            df['estilo'] = df['estilo'].fillna('equilibrado')

        print(f'Base carregada:   {caminho_arquivo}')
        return df
    except:
        print(f'Falha ao ler o arquivo:    {caminho_arquivo}')
        return None
    

base_premier = carregar_liga('premier.csv')
base_laliga = carregar_liga('laliga.csv')
base_bundesliga = carregar_liga('bundesliga.csv')
base_serieA = carregar_liga('seriaA.csv')
base_ligue1 = carregar_liga('ligue1.csv')

Base carregada:   premier.csv
Base carregada:   laliga.csv
Base carregada:   bundesliga.csv
Base carregada:   seriaA.csv
Base carregada:   ligue1.csv


In [3]:
PESO_CONSISTENCIA = 0.10
PESO_FATOR_CASA   = 0.10
PESO_MOMENTO      = 0.05
MEDIA_MOMENTO     = 5
NUM_SIMULACOES    = 150000

In [4]:
RIVAIS = {
    'realmadrid': ['barcelona', 'atletico madrid'],
    'barcelona': ['realmadrid', 'espanyol'],
    'atletico madrid': ['realmadrid'],

    'mancity': ['manutd', 'liverpool', 'arsenal'],
    'manutd': ['mancity', 'liverpool', 'leeds'],
    'liverpool': ['manutd', 'everton'],
    'arsenal': ['tottenham', 'chelsea'],
    'tottenham': ['arsenal', 'chelsea'],
    'chelsea': ['arsenal', 'tottenham'],
    
    'inter': ['milan', 'juventus'],
    'milan': ['inter', 'juventus'],
    'juventus': ['inter', 'milan', 'torino'],
    'roma': ['lazio'],
    'lazio': ['roma'],

    'bayern': ['dortmund'],
    'dortmund': ['bayern', 'schalke'],

    'psg': ['marseille'],
    'marseille': ['psg', 'lyon']
}

In [5]:
CRAQUES = {
    'mancity': [('Haaland', 0.25), ('Cherki', 0.09), ('Semenyo', 0.10)],
    'arsenal': [('Saka', 0.12), ('Gyokeres', 0.10), ('Martinelli', 0.07), ('Jesus', 0.10)],
    'liverpool': [('Ekitike', 0.17), ('Wirtz', 0.8), ('Isak', 0.5)],
    'realmadrid': [('Mbappe', 0.30), ('Vinicius Jr', 0.12), ('Bellingham', 0.09)],
    'barcelona': [('Fermin', 0.10), ('Yamal', 0.10), ('Raphinha', 0.15), ('Ferran', 0.10)],
    'psg': [('Barcola', 0.15), ('Dembele', 0.15)],
    'inter': [('Lautaro', 0.20), ('Thuram', 0.9)],
    'milan': [('Gimenez', 0.11), ('Pulisic', 0.11)],
    'bayern': [('Kane', 0.35), ('Olise', 0.25), ('Diaz', 0.15)]
}

In [6]:
def prever_destaques(time_nome, xg_previsto):
    if time_nome not in CRAQUES:
        return
    
    print(f'   👤 Chances Individuais ({time_nome.upper()}):')
    lista_jogadores = CRAQUES[time_nome]

    for jogador, fatia_gols in lista_jogadores:
        xg_jogador = xg_previsto * fatia_gols
        
        prob_gol = (1 - np.exp(-xg_jogador)) * 100

        if prob_gol > 15:
            emoji = "⚽🔥" if prob_gol > 40 else "👟"
            print(f"      {emoji} {jogador}: {prob_gol:.1f}%")
    print("-" * 30)

In [7]:
def analisar_partida_comp(time_casa, time_fora, df):
    try:
        tc = df[df['time'] == time_casa].iloc[0]
        tf = df[df['time'] == time_fora].iloc[0]

        eh_derby = False
        if time_casa in RIVAIS and time_fora in RIVAIS[time_casa]:
            eh_derby = True

        num_jogos_total = tc['jogos']

        num_jogos_casa = tc['jogcasa']
        num_jogos_fora = tc['jogfora']

        media_gol_jgo = df['gm'].mean() / num_jogos_total

        atk_casa_idx = (tc['gmc'] / num_jogos_casa) / media_gol_jgo
        def_fora_idx = (tf['gsf'] / num_jogos_fora) / media_gol_jgo
        xg_casa = atk_casa_idx * def_fora_idx * media_gol_jgo

        atk_fora_idx = (tf['gmf'] / num_jogos_fora) / media_gol_jgo
        def_casa_idx = (tc['gsc'] / num_jogos_casa) / media_gol_jgo
        xg_fora = atk_fora_idx * def_casa_idx * media_gol_jgo

        media_chutes = df['chutes_jg'].mean()

        fator_chute_c = tc['chutes_jg'] / media_chutes
        fator_chute_f = tf['chutes_jg'] / media_chutes
        xg_casa *= (1 + (fator_chute_c -1) * 0.20)
        xg_fora *= (1 + (fator_chute_f-1) * 0.20)

        aprov_casa = (tc['vitorias']*3 + tc['empates']) / (tc['jogos']*3)
        aprov_fora = (tf['vitorias']*3 + tf['empates']) / (tf['jogos']*3)
        xg_casa *= (1 + (aprov_casa * PESO_CONSISTENCIA))
        xg_fora *= (1 + (aprov_fora * PESO_CONSISTENCIA))

        peso_casa_efetivo = PESO_FATOR_CASA

        if eh_derby:
            peso_casa_efetivo = PESO_FATOR_CASA / 3
            print(f"   ⚔️ ALERTA DE CLÁSSICO! Vantagem de casa reduzida.")


        aprov_mandante = (tc['vitcasa']*3 + tc['empcasa']) / (num_jogos_casa*3)
        aprov_visitante = (tf['vitfora']*3 + tf['empfora']) / (num_jogos_fora*3)
        xg_casa *= (1 + (aprov_mandante * peso_casa_efetivo))
        xg_fora *= (1 + (aprov_visitante * PESO_FATOR_CASA))

        momento_c = 1 + ((tc['perf_ult_5_jgs'] - MEDIA_MOMENTO) * PESO_MOMENTO)
        momento_f = 1 + ((tf['perf_ult_5_jgs'] - MEDIA_MOMENTO) * PESO_MOMENTO)
        xg_casa *= momento_c
        xg_fora *= momento_f

        est_c = tc['estilo']
        est_f = tf['estilo']
        mult_c, mult_f = 1.0, 1.0

        print(f'========================================')
        print(f'🏟️  DUELO: {time_casa.upper()} ({est_c}) x {time_fora.upper()} ({est_f})')

        if est_c == 'posse' and est_f == 'contra_ataque':
            mult_f += 0.15
            print('   ⚠️ TÁTICA: Contra-ataque visitante explora a Posse.')
        elif est_c == 'contra_ataque' and est_f == 'posse': 
            mult_c += 0.15
            print('   ⚠️ TÁTICA: Contra-ataque mandante explora a Posse.')
        if est_c == 'defensivo' and est_f == 'pressao':
            mult_f += 0.15
            print("   ⚠️ TÁTICA: Pressão alta visitante quebra a Retranca.")
        elif est_c == 'pressao' and est_f == 'defensivo':
            mult_c += 0.15
            print('   ⚠️ TÁTICA: Pressão alta mandante quebra a Retranca.')
        if est_c == 'defensivo' and est_f == 'contra_ataque':
            mult_c += 0.10
            print('   🛡️ TÁTICA: Retranca anula velocidade do visitante.')
        elif est_c == 'contra_ataque' and est_f == 'defensivo':
            mult_f += 0.10
            print('   🛡️ TÁTICA: Retranca visitante anula velocidade da casa.')

        if eh_derby:
            mult_c = 1 + (mult_c - 1) * 0.5
            mult_f = 1 + (mult_f - 1) * 0.5
            print("   ⚔️ FATOR DERBY: Influência tática reduzida pela metade (Jogo Nervoso).")

        xg_casa_final = xg_casa * mult_c
        xg_fora_final = xg_fora * mult_f

        if eh_derby:
            xg_medio = (xg_casa_final + xg_fora_final) / 2

            xg_casa_final = (xg_casa_final * 0.60) + (xg_medio * 0.40)
            xg_fora_final = (xg_fora_final * 0.60) + (xg_medio * 0.40)
            
            print(f"   ⚖️ NIVELADOR ATIVADO: Forças aproximadas em 40% pela rivalidade.")

        gols_simu_casa = np.random.poisson(xg_casa_final, NUM_SIMULACOES)
        gols_simu_fora = np.random.poisson(xg_fora_final, NUM_SIMULACOES)

        vitorias_c = np.sum(gols_simu_casa > gols_simu_fora)
        vitorias_f = np.sum(gols_simu_fora > gols_simu_casa)
        empates = np.sum(gols_simu_casa == gols_simu_fora)

        prob_c = (vitorias_c / NUM_SIMULACOES) *100
        prob_f = (vitorias_f / NUM_SIMULACOES) *100
        prob_e = (empates / NUM_SIMULACOES) *100

        print(f'========================================')
        print(f'🏟️  PREVISÃO: {time_casa.upper()} x {time_fora.upper()}')
        print(f'========================================')
        print(f'📊 DADOS TÉCNICOS:')
        print(f'   > xG (Gols Esperados): {time_casa} {xg_casa_final:.2f} - {xg_fora_final:.2f} {time_fora}')
        print(f"   > Fase (0-10): {time_casa} nota {tc['perf_ult_5_jgs']} | {time_fora} nota {tf['perf_ult_5_jgs']}")
        
        print(f'\n🎲 PROBABILIDADES (Baseado em {NUM_SIMULACOES} simulações):')
        print(f'   🟢 Vitória {time_casa}: {prob_c:.1f}%')
        print(f'   ⚪ Empate: {prob_e:.1f}%')
        print(f'   🔴 Vitória {time_fora}: {prob_f:.1f}%')
        
        print(f'\n💡 CONCLUSÃO DA IA:')
        if prob_c > 60:
            print(f'  Favorito claro: {time_casa.upper()}')
            print(f'   (Mas cuidado: Existe {prob_f+prob_e:.1f}% de chance de tropeço)')
        elif prob_f > 60:
            print(f'   Favorito Claro: {time_fora.upper()}.')
            print(f'   (Mas cuidado: Existe {prob_c+prob_e:.1f}% de chance de tropeço)')
        else:
            print(f'   JOGO ABERTO! As chances estão divididas.')
            if abs(prob_c - prob_f) < 5:
                print('  Recomendação: Evitar apostar em vencedor (Mercado de Gols ou Empate é melhor).')

        return float(xg_casa_final), float(xg_fora_final)
    
    except IndexError:
        print(f"❌ Erro: Time '{time_casa}' ou '{time_fora}' não encontrado no DataFrame.")
    except Exception as e:
        print(f"❌ Erro técnico: {e}")

        return 0.0, 0.0

In [8]:
xg1, xg2 = analisar_partida_comp('mancity', 'manutd', base_premier)

prever_destaques('mancity', xg1)

   ⚔️ ALERTA DE CLÁSSICO! Vantagem de casa reduzida.
🏟️  DUELO: MANCITY (posse) x MANUTD (contra_ataque)
   ⚠️ TÁTICA: Contra-ataque visitante explora a Posse.
   ⚔️ FATOR DERBY: Influência tática reduzida pela metade (Jogo Nervoso).
   ⚖️ NIVELADOR ATIVADO: Forças aproximadas em 40% pela rivalidade.
🏟️  PREVISÃO: MANCITY x MANUTD
📊 DADOS TÉCNICOS:
   > xG (Gols Esperados): mancity 3.13 - 1.70 manutd
   > Fase (0-10): mancity nota 5 | manutd nota 8

🎲 PROBABILIDADES (Baseado em 150000 simulações):
   🟢 Vitória mancity: 66.0%
   ⚪ Empate: 15.5%
   🔴 Vitória manutd: 18.4%

💡 CONCLUSÃO DA IA:
  Favorito claro: MANCITY
   (Mas cuidado: Existe 34.0% de chance de tropeço)
   👤 Chances Individuais (MANCITY):
      ⚽🔥 Haaland: 54.3%
      👟 Cherki: 24.5%
      👟 Semenyo: 26.9%
------------------------------


In [10]:
xg1, xg2 = analisar_partida_comp('realmadrid', 'barcelona', base_laliga)


   ⚔️ ALERTA DE CLÁSSICO! Vantagem de casa reduzida.
🏟️  DUELO: REALMADRID (posse) x BARCELONA (posse)
   ⚔️ FATOR DERBY: Influência tática reduzida pela metade (Jogo Nervoso).
   ⚖️ NIVELADOR ATIVADO: Forças aproximadas em 40% pela rivalidade.
🏟️  PREVISÃO: REALMADRID x BARCELONA
📊 DADOS TÉCNICOS:
   > xG (Gols Esperados): realmadrid 4.02 - 2.50 barcelona
   > Fase (0-10): realmadrid nota 10 | barcelona nota 8

🎲 PROBABILIDADES (Baseado em 150000 simulações):
   🟢 Vitória realmadrid: 65.5%
   ⚪ Empate: 13.5%
   🔴 Vitória barcelona: 21.0%

💡 CONCLUSÃO DA IA:
  Favorito claro: REALMADRID
   (Mas cuidado: Existe 34.5% de chance de tropeço)
